In [1]:
import numpy as np
from statsmodels.tsa.arima_model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX
import statsmodels.api as sm
import time

In [2]:
train = np.load("../data/train.npy")
val = np.load("../data/val.npy")
mini = np.load("../data/mini_train.npy")

In [3]:
print(mini.shape)

(28, 100, 100)


In [25]:
p, d, q = 1, 1, 1

window_size = train.shape[-1]

print(f"window_size: {window_size}")
fitted_models = []
for i in range(window_size**2):
    start_time = time.time()

    print(f"training model: {i+1}/{window_size**2 + 1}")
    model = ARIMA(train[:, i // window_size, i % window_size], order=(p, d, q))
    model_fit = model.fit(disp=0)
    fitted_models.append(model_fit)
#     model_fit.save(f"temp/save_{i}.pickle")
    
    elapsed = time.time() - start_time
    print(f"took: {elapsed} seconds")
    
    break

window_size: 100
training model: 1/10001
took: 0.8920671939849854 seconds


In [39]:
model = fitted_models[0]
model.predict(start=10, end=20, exog=np.random.randn(100)).shape

(11,)

In [19]:
for i in range(8):
    start_time = time.time()
    print(f"loading model")
    saved = sm.load(f"temp/save_{i}.pickle")
    
    elapsed = time.time() - start_time
    print(f"took: {elapsed} seconds")

loading model
took: 0.0029108524322509766 seconds
loading model
took: 0.0020389556884765625 seconds
loading model
took: 0.0020127296447753906 seconds
loading model
took: 0.0020232200622558594 seconds
loading model
took: 0.0015790462493896484 seconds
loading model
took: 0.002247333526611328 seconds
loading model
took: 0.0019788742065429688 seconds
loading model
took: 0.0015597343444824219 seconds


In [ ]:
def predict(coef, history):
    yhat = 0.0
    for i in range(1, len(coef)+1):
        yhat += coef[i-1] * history[-i]
    return yhat
 
def difference(dataset):
    diff = list()
    for i in range(1, len(dataset)):
        value = dataset[i] - dataset[i - 1]
        diff.append(value)
    return numpy.array(diff)
 
def predict(model_fit, orders, inputs, output_length)
# series = Series.from_csv('daily-minimum-temperatures.csv', header=0)
# X = series.values
# size = len(X) - 7
# train, test = X[0:size], X[size:]
# history = [x for x in train]
# predictions = list()
for t in range(len(test)):

    ar_coef, ma_coef = model_fit.arparams, model_fit.maparams
    resid = model_fit.resid
    for i in range(d):
        inputs = difference(inputs)
        
    
    yhat = history[-1] + predict(ar_coef, diff) + predict(ma_coef, resid)
    predictions.append(yhat)
    obs = test[t]
    history.append(obs)
    print('>predicted=%.3f, expected=%.3f' % (yhat, obs))
rmse = sqrt(mean_squared_error(test, predictions))
print('Test RMSE: %.3f' % rmse)

In [43]:
mod = SARIMAX(train[:50, 1, 1], order=(1,1,1))
mod_fit = mod.fit()

/Users/dziugas/anaconda3/envs/msc/lib/python3.6/site-packages/statsmodels/base/model.py:512: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)


In [46]:
naujas = SARIMAX(train[50:100, 1, 1], order=(1,1,1))
naujas_fit = naujas.filter(mod_fit.params)

In [50]:
pred = naujas_fit.get_prediction(start=10, end=14, dynamic=True)

In [54]:
pred.predicted_mean

array([-0.45269765, -0.4576199 , -0.45877185, -0.45904144, -0.45910453])

In [92]:
naujas_fit.params

array([ 2.34030314e-01, -8.47683842e-01,  3.41470988e-05])

In [61]:
naujas_fit.save("temp.pickle", remove_data=True)

In [8]:
def nrmse(targets, predictions):
    targets = np.squeeze(targets)
    predictions = np.squeeze(predictions)

    error = predictions - targets
    mse = np.mean(np.square(error))
    rmse = np.sqrt(mse)
    return rmse / np.mean(targets)

def make_prediction(save_path, eval_data, order, input_start, input_size, output_size):
    grid_size = 100
    prediction = np.zeros((output_size, grid_size, grid_size))
    for x_coord in range(grid_size):
        print(f"x_coord: {x_coord}")
        for y_coord in range(grid_size):
            
            trained_model = sm.load(f"{save_path}/{x_coord}_{y_coord}.pickle")
            model = SARIMAX(eval_data[:, x_coord, y_coord], order=order)
            model_fit = model.filter(trained_model.params)
            
            prediction_wrapper = model_fit.get_prediction(start=input_start, 
                                     end=input_start + input_size + output_size - 1, dynamic=input_size)
            

#             print(f"pred mean: {prediction_wrapper.predicted_mean}")
            prediction[:, x_coord, y_coord] += prediction_wrapper.predicted_mean[-output_size:]
#             print(prediction[:, x_coord, y_coord])
            
            
    return prediction

def evaluate(save_path, eval_data, order, output_size=12, input_size=12,
             train_mean=67.61768898039853, train_std=132.47248595705986):
#     input_size = order[0]  # p is input_size
    errors = []
    for i in range(len(eval_data) - output_size - input_size):
        predictions = make_prediction(save_path, eval_data, order, i, input_size, output_size)
        targets = eval_data[i+input_size:i+input_size+output_size]
        
        predictions = predictions * train_std + train_mean 
        targets = targets * train_std + train_mean
        
        error = nrmse(targets, predictions)
        print(f"error: {error}")
        errors.append(error)
        break
            
    print(f"mean error: {np.array(errors).mean()}")
    print(f"error std: {np.array(errors).std()}")
    return errors
    

            

In [9]:
save_path = "../results/arima/p1_d0_q0/saved_models"

errors = evaluate(save_path, val, order=(1, 0, 0))

x_coord: 0
x_coord: 1
x_coord: 2
x_coord: 3
x_coord: 4
x_coord: 5
x_coord: 6
x_coord: 7
x_coord: 8
x_coord: 9
x_coord: 10
x_coord: 11
x_coord: 12
x_coord: 13
x_coord: 14
x_coord: 15
x_coord: 16
x_coord: 17
x_coord: 18
x_coord: 19
x_coord: 20
x_coord: 21
x_coord: 22
x_coord: 23
x_coord: 24
x_coord: 25
x_coord: 26
x_coord: 27
x_coord: 28
x_coord: 29
x_coord: 30
x_coord: 31
x_coord: 32
x_coord: 33
x_coord: 34
x_coord: 35
x_coord: 36
x_coord: 37
x_coord: 38
x_coord: 39
x_coord: 40
x_coord: 41
x_coord: 42
x_coord: 43
x_coord: 44
x_coord: 45
x_coord: 46
x_coord: 47
x_coord: 48
x_coord: 49
x_coord: 50
x_coord: 51
x_coord: 52
x_coord: 53
x_coord: 54
x_coord: 55
x_coord: 56
x_coord: 57
x_coord: 58
x_coord: 59
x_coord: 60
x_coord: 61
x_coord: 62
x_coord: 63
x_coord: 64
x_coord: 65
x_coord: 66
x_coord: 67
x_coord: 68
x_coord: 69
x_coord: 70
x_coord: 71
x_coord: 72
x_coord: 73
x_coord: 74
x_coord: 75
x_coord: 76
x_coord: 77
x_coord: 78
x_coord: 79
x_coord: 80
x_coord: 81
x_coord: 82
x_coord: 83
x_